In [1]:
import time 
import json 
import random 
from datetime import datetime
from data_generator import generate_message
from kafka import KafkaProducer
import alpaca_trade_api as tradeapi
from alpaca_trade_api.stream import Stream
from alpaca_trade_api.common import URL
import threading
import websocket
import pandas as pd

data = pd.read_json("dummy_new") 
print(data)

                           Date    Open    High     Low   Close  Volume  \
0     2015-12-14 13:15:00+00:00  111.25  111.70  111.20  111.43  104293   
1     2015-12-14 13:30:00+00:00  111.53  111.85  111.35  111.85   82679   
2     2015-12-14 13:45:00+00:00  111.89  112.25  111.80  111.86  188722   
3     2015-12-14 14:00:00+00:00  111.86  112.00  111.60  111.70   85606   
4     2015-12-14 14:15:00+00:00  111.69  112.13  111.68  112.00  132055   
...                         ...     ...     ...     ...     ...     ...   
90946 2021-11-10 23:45:00+00:00  148.10  148.10  147.90  147.92   20713   
90947 2021-11-11 00:00:00+00:00  147.98  148.03  147.94  148.03   12868   
90948 2021-11-11 00:15:00+00:00  148.02  148.19  148.01  148.16   12056   
90949 2021-11-11 00:30:00+00:00  148.18  148.28  148.15  148.23   21241   
90950 2021-11-11 00:45:00+00:00  148.22  148.24  148.13  148.15   20195   

       Trade_count        vwap  
0              373  111.499108  
1              313  111.584751  


In [ ]:


# Messages will be serialized as JSON 
def serializer(message):
    return json.dumps(message).encode('utf-8')


# Kafka Producer
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=serializer
)

data_url = 'wss://data.alpaca.markets'
base_url = 'https://paper-api.alpaca.markets'
api_key= 'PKZEXWB0J5CEU2JJZXV8'
api_secret = 'kPy7mGsSnzeFnTgpMKFAQG58blbwqYgPiTeEeUPo'

def alpaca():
    def on_open(ws):
        print("opened")
        auth_data = {"action": "auth", "key": api_key, "secret": api_secret}
        print(auth_data)
        ws.send(json.dumps(auth_data))
        listen_message = {"action": "subscribe", "bars": ["AAPL"]}       
        ws.send(json.dumps(listen_message))


    def on_message(ws, message):        
        allowed=['b']
        #print(f'Sending @ {datetime.now()} | Message = {str(message)[1:-1]}')
        msgjson = json.loads(message[1:-1])
        if(msgjson['T'] in allowed):  
            print(msgjson)
            body = {
                "Date": str(msgjson["t"]),
                "Open": str(msgjson["o"]),             
                "High": str(msgjson["h"]),
                "Low": str(msgjson["l"]),
                "Close": str(msgjson["c"]),    
                "Volume": str(msgjson["v"]),
                "Trade_count": str(msgjson["n"]),
                "vwap": str(msgjson["vw"])
            }            
            producer.send('messages', body)
        
        # Sleep for a random number of seconds
        #time_to_sleep = random.randint(1, 11)
        time.sleep(2)

    def on_close(ws, close_status, closemessage):
        print("closed connection ", closemessage)        
        print ("Retry : %s" % time.ctime())
        time.sleep(1)
        connect_websocket() # retry per 10 seconds

    def on_error(ws, message):
        print("ERROR connection ", message)

    def connect_websocket():
        socket = "wss://stream.data.alpaca.markets/v2/iex"
        ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_close=on_close, on_error=on_error)
        ws.run_forever()

    connect_websocket()

def aftermarket():   
    for index, row in data.iterrows():  
        #print(json.dumps(json.loads(row.to_json(date_format = "iso",orient = "columns"))))
        producer.send('messages', json.loads(row.to_json(date_format = "iso")))
        time.sleep(300)
    #csv_file = pd.DataFrame(pd.read_csv("apple_data.csv", sep = ",", header = 0, index_col = False))
    #csv_file.to_json("dummy_new", orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)
    
    #while(1):
#     with open('dummy_new') as f:
#         lines = f.readlines()
#         for line in lines:              
#             print(json.loads(line))
#             #producer.send('messages', json.loads(line))
#             # Sleep for a random number of seconds
#             time_to_sleep = random.randint(1, 10)
#             time.sleep(3)


if __name__ == '__main__':

    api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')
    clock = api.get_clock()
    if clock.is_open:
        alpaca()
    else:
        aftermarket()

    